<a href="https://colab.research.google.com/github/nakkata/dev_tool/blob/main/chat_gpt_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# チャットGPTで社内ドキュメントの解析エンジンを作る

In [ ]:
!pip install openai
!pip install pandas
!pip install transformers
!pip install langchain

In [ ]:
#!pip install codecs
!pip install textract
!pip install beautifulsoup4
!pip install requests

In [3]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 3.2 MB/s eta 0:00:00


In [4]:
!pip install tiktoken
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.8 MB/s eta 0:00:00


In [5]:
import os
import pandas as pd
import requests
import textract
import codecs
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [6]:

# ここにOpenAIから取得したキーを設定します。
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
# 社内用で使う時はここをコメントアウト
## proxyの設定
#os.environ["HTTP_PROXY"] = "http://10.40.210.120:8080"
#os.environ["HTTPS_PROXY"] = "http://10.40.210.120:8080"

In [7]:

# LLMの設定
llm = OpenAI(model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [8]:

# インターネット上からマニュアルファイルをダウンロード（BuffaloTerastation)
url = 'https://support2.mexcbt.mext.go.jp/files/MEXCBT簡易マニュアル_児童編（OPE）_第4版.pdf'
response = requests.get(url)

In [9]:
# Ensure that the request was successful
if response.status_code == 200:
    # Save the content of the response as a PDF file
    with open('sample_document1.pdf', 'wb') as file:
        file.write(response.content)
else:
    print("Error: Unable to download the PDF file. Status code:", response.status_code)

In [10]:
loader = PyPDFLoader("/content/sample_document1.pdf")
pages = loader.load_and_split()

In [11]:
chunks = pages
print("step2")

step2


In [12]:

# Get embedding model
embeddings = OpenAIEmbeddings()


In [13]:

#  vector databaseの作成
db = FAISS.from_documents(chunks, embeddings)

In [ ]:
query = "自治体コード入力画面とは？"
# FAISSに対して検索。検索は文字一致ではなく意味一致で検索する(Vector, Embbeding)
docs = db.similarity_search(query)
docs # ここで関係のありそうなデータが返ってきていることを確認できる


In [15]:
print("step7")
# 得られた情報から回答を導き出すためのプロセスを以下の4つから選択する。いずれもProsとConsがあるため、適切なものを選択する必要がある。
# staffing ... 得られた候補をそのままインプットとする
# map_reduce ... 得られた候補のサマリをそれぞれ生成し、そのサマリのサマリを作ってインプットとする
# map_rerank ... 得られた候補にそれぞれスコアを振って、いちばん高いものをインプットとして回答を得る
# refine  ... 得られた候補のサマリを生成し、次にそのサマリと次の候補の様裏を作ることを繰り返す
chain = load_qa_chain(OpenAI(temperature=0.1,max_tokens=1000), chain_type="stuff")
# p305に記載
#query = "ドライブのランプが赤色に点滅しているが、これは何が原因か？"
# p134に記載
#query = "どの様な時にメイン機が異常だと判断をしますか？"
query = "ログイン方法を教えてください。"
docs = db.similarity_search(query)
print("step8")
# langchainを使って検索
chain.run(input_documents=docs, question=query)


step7
step8


' 先生から聞いた情報を入力し、自治体コード入力画面で自治体コードを入力し、次に学校名・学年・ＩＤ・パスワードを入力し、［ログイン］をクリックすると、ポータルシステムへログインされ、ポータルシステム画面が表示されます。'

In [16]:
from IPython.display import display
import ipywidgets as widgets

In [20]:

print("step9")
# vextordbをretrieverとして使うconversation chainを作成します。これはチャット履歴の管理も可能にします。
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

step9


In [21]:
chat_history = []

In [22]:
print("step10")
def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("Thank you for using the State of the Union chatbot!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Transformers chatbot! Type 'exit' to stop.")


step10
Welcome to the Transformers chatbot! Type 'exit' to stop.


In [23]:
input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Text(value='', placeholder='Please enter your question:')

HTML(value='<b>User:</b> MEXCBTとは何ですか')

HTML(value='<b><font color="blue">Chatbot:</font></b>  MEXCBTは、メクビット学習システムのスタートアップガイドで、児童向けの学習プログラムです。')